In [92]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [93]:
def RolarTravaAltaPut(ativosLista, comprei, vendi ):
    # [travaFeitaC (comprada) , travaFeitaV (vendida), travaNovaC, travaNovaV]
    travaFeitaC = mt5.symbol_info(ativosLista[0])
    travaFeitaV = mt5.symbol_info(ativosLista[1])
    travaNovaC  = mt5.symbol_info(ativosLista[2])
    travaNovaV  = mt5.symbol_info(ativosLista[3])
    # ativoBase = travaFeitaC.basis
    ativoBaseInfo = mt5.symbol_info(travaFeitaC.basis)

    # gerar matriz
    lin,col = (5,9)
    matriz = [[0 for i in range(col)] for j in range(lin)]
    ###########################################
    # coluna 0 - ativos
    matriz[0][0] = ativoBaseInfo.name
    matriz[1][0] = travaFeitaC.name 
    matriz[2][0] = travaFeitaV.name
    matriz[3][0] = 'dif'
    matriz[4][0] = '%'

    # coluna 1 - strike feitos
    matriz[0][1] = ativoBaseInfo.last
    matriz[1][1] = travaFeitaC.option_strike
    matriz[2][1] = travaFeitaV.option_strike
    matriz[3][1] = travaFeitaV.option_strike - travaFeitaC.option_strike
    matriz[4][1] = (travaFeitaV.option_strike - travaFeitaC.option_strike -1) 

    # coluna 2 - valores da trava feita
    matriz[0][2] = '*'
    matriz[1][2] = round(comprei,2)
    matriz[2][2] = round(vendi,2)
    matriz[3][2] = round(matriz[1][2] + matriz[2][2], 2)

    # coluna 3 - valores para desmontar a trava feita
    matriz[0][3] = '**'
    matriz[1][3] = round(travaFeitaC.bid,2)
    matriz[2][3] = round(-travaFeitaV.ask,2)
    matriz[3][3] = round(matriz[1][3] + matriz[2][3], 2)

    # coluna 4 - ativos novos
    matriz[0][4] = '***'
    matriz[1][4] = travaNovaC.name
    matriz[2][4] = travaNovaV.name
    matriz[3][4] = 'dif'
    matriz[4][4] = '%'

    
    # coluna 5 - strikes novos
    matriz[0][5] = '****'
    matriz[1][5] = round(travaNovaC.option_strike,2)
    matriz[2][5] = round(travaNovaV.option_strike,2)
    matriz[3][5] = matriz[2][5] - matriz[1][5]

    # coluna 6 - valores para montar nova trava
    matriz[0][6] = '****'
    matriz[1][6] = -travaNovaC.ask
    matriz[2][6] = travaNovaV.bid
    if matriz[2][6] == 0:
        fff = input('Entre o valor do Bid ' + travaNovaV.name + ': ')
        matriz[2][6] = float(fff)
    matriz[3][6] = matriz[1][6] + matriz[2][6]

    # COLUNA 7 - LAST
    matriz[0][7] ='.'
    matriz[1][7] = -travaFeitaC.last
    matriz[2][7] = travaFeitaV.last
    matriz[3][7] = matriz[1][7] + matriz[2][7]
    matriz[4][7] = '' 

    # COLUNA 8 - Black and Sholder
    matriz[0][8] ='..'
    matriz[1][8] = 22
    matriz[2][8] = 33
    matriz[3][8] = matriz[1][8] + matriz[2][8]
    matriz[4][8] = '' 
   

    df = pd.DataFrame(matriz, columns = ['Ativo', '$$$', '$ Montei', '$ Desmontar' , 'ativo', 'Strike Novo', 'Montar Novo',  '$ last' , 'BS'])
    print(df)
    #nome_arq = 'RolarTravaAltaPut_' + ativoBase  + '.xlsx'
    #df.to_excel(nome_arq)


In [94]:
RolarTravaAltaPut(['BBSES220', 'BBSES235', 'BBSET220', 'BBSET235'], -0.15, 0.4)

      Ativo    $$$ $ Montei $ Desmontar     ativo Strike Novo Montar Novo  \
0     BBSE3  22.67        *          **       ***        ****        ****   
1  BBSES220  22.00    -0.15        0.08  BBSET220        22.0        -0.4   
2  BBSES235  23.50      0.4        -0.0  BBSET235        23.5        0.95   
3       dif   1.50     0.25        0.08       dif         1.5        0.55   
4         %   0.50        0           0         %           0           0   

  $ last  BS  
0      .  ..  
1  -0.11  22  
2   0.91  33  
3    0.8  55  
4             
